In [1]:
%pwd

'd:\\recommandation\\MLproject\\research'

In [2]:
%cd ..

d:\recommandation\MLproject


d:\recommandation\rithvik\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataPreprocessConfig:
    root_dir: Path
    data_path: Path
    cat_params: list
    drop_params: list

In [4]:
lib = %pwd

In [5]:
import sys

sys.path.append(lib)

In [6]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories
from src.mlProject import logger

In [7]:
from pathlib import Path

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE,
        params_filepath = PARAMS_FILE):

        config_filepath = Path(config_filepath)
        params_filepath = Path(params_filepath)

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
        
    def get_data_preprocessing_config(self) -> DataPreprocessConfig:
        config = self.config.data_preprocessing
        drop_columns = self.params['features_to_drop']
        cat_columns = self.params['categorical_features']
        
        create_directories([config.root_dir])
        data_preprocessing_config = DataPreprocessConfig(
            root_dir = config.root_dir,
            data_path = config.data_dir,
            cat_params = cat_columns,
            drop_params = drop_columns  # Pass DROP_COLUMNS to DataPreprocessConfig
        )
        
        return data_preprocessing_config


In [8]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import os
import logging  

class DataPreprocessing:
    def __init__(self, config: DataPreprocessConfig):
        self.config = config
        
    def _read_data(self):
        try:
            # Read data from CSV
            self.df = pd.read_csv(self.config.data_path)
        except Exception as e:
            logging.error(f"Error occurred while reading data: {str(e)}")
            raise e
    
    def _calculate_delivery_time(self):
        try:
            self.df['order_purchase_timestamp'] = pd.to_datetime(self.df['order_purchase_timestamp'])
            self.df['order_delivered_customer_date'] = pd.to_datetime(self.df['order_delivered_customer_date'])

            # Now that both columns are in datetime format, you can perform the subtraction
            self.df['Time_taken_for_delivery'] = (self.df['order_delivered_customer_date'] - self.df['order_purchase_timestamp']).dt.days
        except Exception as e:
            logging.error(f"Error occurred while calculating delivery time: {str(e)}")
            raise e
            
    def _calculate_delivery_delay(self):
        try:
            self.df['order_estimated_delivery_date'] = pd.to_datetime(self.df['order_estimated_delivery_date'])
            self.df['delivery_delay'] = (self.df['order_delivered_customer_date'] - self.df['order_estimated_delivery_date']).dt.days
        except Exception as e:
            logging.error(f"Error occurred while calculating delivery delay: {str(e)}")
            raise e
    
    def _encode_categorical_data(self):
        try:
            self.df_encoded = pd.get_dummies(self.df, columns=self.config.cat_params)
        except Exception as e:
            logging.error(f"Error occurred while encoding categorical data: {str(e)}")
            raise e
    
    def _scale_data(self):
        try:
            scaler = MinMaxScaler()

            # Reshape the 'review_score' column to a 2D array
            review_score = self.df_encoded['delivery_delay'].values.reshape(-1, 1)

            # Fit and transform the data
            scaled_review_score = scaler.fit_transform(review_score)

            # Replace the original 'review_score' column with the scaled values
            self.df_encoded['delivery_delay'] = scaled_review_score
        except Exception as e:
            logging.error(f"Error occurred while scaling data: {str(e)}")
            raise e
            
    def _drop_columns(self):
        try:
            self.df_encoded = self.df_encoded.drop(columns=self.config.drop_params)
        except Exception as e:
            logging.error(f"Error occurred while dropping columns: {str(e)}")
            raise e
    
    def _save_preprocessed_data(self):
        try:
            output_file_path = os.path.join(self.config.root_dir, "preprocessed_data.csv")
            self.df_encoded.to_csv(output_file_path, index=False)
            logging.info(f"Preprocessed data saved to: {output_file_path}")  
        except Exception as e:
            logging.error(f"Error occurred while saving preprocessed data: {str(e)}")
            raise e
    
    def preprocess_data(self):
        try:
            self._read_data()
            self._calculate_delivery_time()
            self._calculate_delivery_delay()
            self._encode_categorical_data()
            self._scale_data()
            self._drop_columns()
            self._save_preprocessed_data()
        except Exception as e:
            logging.error(f"Error occurred during data preprocessing: {str(e)}")
            raise e


In [9]:
try:
    config = ConfigurationManager()
    data_preprocessing_config = config.get_data_preprocessing_config()
    data_preprocessing = DataPreprocessing(config=data_preprocessing_config)
    data_preprocessing.preprocess_data()
except Exception as e:
    raise e


[2024-05-11 18:10:00,817: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-11 18:10:00,837: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-11 18:10:00,839: INFO: common: created directory at: artifacts]
[2024-05-11 18:10:00,841: INFO: common: created directory at: artifacts/data_preprocessing]
[2024-05-11 18:15:51,502: INFO: 2784740437: Preprocessed data saved to: artifacts/data_preprocessing\preprocessed_data.csv]
